# Even / Odd training denoising of corbel_big

* Inputs: `even.mrc` and `odd.mrc`, the pair of vol vols.
* Outputs: `even_odd_denoised/denoised.mrc` with the denoised vol, and `denoised_even_odd.pdf` with a view of a tile of a central slice in Z.

In [ ]:
from pathlib import Path
from collections import namedtuple

In [ ]:
Args = namedtuple("args", ["even", "odd", "denoised"])
args = Args("even.mrc", "odd.mrc", "even_odd_denoised/vol.mrc")

In [ ]:
if Path(args.denoised).exists():
    raise Exception(f"{args.denoised} already exists ... exiting")

In [ ]:
if not Path("even.mrc").exists() or not Path("odd.mrc"):
    %run split_even_odd.ipynb

In [ ]:
from my_google_auth import DriveHandler
service = DriveHandler.get_drive_service()
handler = DriveHandler.DriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'

## Configure cryoCARE

In [ ]:
%%writefile train_data_config__evenodd.json
{
    "even": ["even.mrc"],
    "odd": ["odd.mrc"],
    "mask": [""],
    "patch_shape": [16, 16, 16],
    "num_slices": 800,
    "split": 0.9,
    "tilt_axis": "Y",
    "n_normalization_samples": 200,
    "path": "./even_odd_data",
    "overwrite": "True"  
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/empiar10311
source ~/envs/cryoCARE/bin/activate
cryoCARE_extract_train_data.py --conf train_data_config__evenodd.json

## Train

In [ ]:
%%writefile train_config__evenodd.json
{
  "train_data": "./even_odd_data",
  "epochs": 50,
  "steps_per_epoch": 200,
  "batch_size": 16,
  "unet_kern_size": 3,
  "unet_n_depth": 3,
  "unet_n_first": 16,
  "learning_rate": 0.0004,
  "model_name": "model",
  "path": "./",
  "gpu_id": [1]
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/corbel_big
#pwd
source ~/envs/cryoCARE/bin/activate
#cryoCARE_extract_train_data.py --conf train_data_config__evenodd.json
cryoCARE_train.py --conf train_config__evenodd.json

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='model.tar.gz',
    drive_file_name='corbel_small__denoised__cryoCARE_even_odd__model.tar.gz',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

## Infer

In [ ]:
%%writefile predict_config__evenodd.json
{
    "path": "./model.tar.gz",
    "even": ["vol.mrc"], 
    "odd": ["vol.mrc"],
    "n_tiles": [1,1,1],
    "output": "even_odd_denoised",
    "overwrite": "True",
    "gpu_id": [1]
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/corbel_big
pwd
source ~/envs/cryoCARE/bin/activate
cryoCARE_predict.py --conf predict_config__evenodd.json || true

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='even_odd_denoised/vol.mrc',
    drive_file_name='corbel_small__denoised__cryoCARE_even_odd.mrc',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
import mrcfile
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
def read_MRC(file_path):
    return mrcfile.read(file_path)

In [ ]:
mrc_file_path = 'vol.mrc'
original_volume = read_MRC(mrc_file_path)

In [ ]:
original_volume.shape

In [ ]:
mrc_file_path = 'even_odd_denoised/vol.mrc'
denoised_volume = read_MRC(mrc_file_path)

In [ ]:
denoised_volume.shape

In [ ]:
# Choose a slice index in the middle of the volume for a good comparison
slice_idx = original_volume.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(original_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N Even/Odd Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised_volume.shape[0]//2
plt.imshow(denoised_volume[slice_idx, 0:400, 400:800], cmap="gray")
plt.savefig("denoised_even_odd.pdf", bbox_inches='tight')

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoised_even_odd.pdf',
    drive_file_name='corbel_small__denoised__cryoCARE_even_odd.pdf',
    drive_folder_id=MY_SHARED_DRIVE_ID
)